In [6]:
import pygame
from assets_model import AssetsModel

pygame.init()
screen = pygame.display.set_mode((800, 450))
clock = pygame.time.Clock()

model = AssetsModel(base_dir=".")

# 1) 로드 + 2) dict 저장
model.load("player", "assets/player.png")  # kind 자동 추론 (image)
model.load_font_with_path("ui_kr", "C:/Windows/Fonts/malgun.ttf", 24)  # 경로 기반 폰트 로드

# 3) 활용: 크기 조절
player128 = model.image_scaled("player", (128, 128))
playerContain = model.image_fit_contain("player", (200, 100))  # 비율유지-박스내
playerCover   = model.image_fit_cover("player", (200, 100))    # 비율유지-박스가득

font48 = model.font_resized_safe("ui_kr", 48)
txt = font48.render("안녕, 모델 1단계!", True, (240,240,240))

running = True
x = 100
while running:
    dt = clock.tick(144)/1000
    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            running = False

    x += 100*dt

    screen.fill((22,25,34))
    screen.blit(player128, (50, 180))
    screen.blit(playerContain, (220, 180))
    screen.blit(playerCover,   (430, 170))
    screen.blit(txt, (40, 30))
    pygame.display.flip()

pygame.quit()


ImportError: cannot import name 'AssetsModel' from 'assets_model' (d:\_Python\assets_model.py)

In [ ]:
# background.py
# pygame 2.x 기준
import pygame
from typing import List, Tuple, Optional

Vec2 = pygame.math.Vector2

# -------------------------------
# 카메라: 월드 좌표 -> 스크린 좌표 변환의 기준
# -------------------------------
class Camera:
    def __init__(self, pos: Tuple[float, float] = (0, 0)):
        self.pos = Vec2(pos)  # 월드 기준 카메라 위치
        self.vel = Vec2(0, 0) # 선택: 외부에서 제어

    def update(self, dt: float):
        self.pos += self.vel * dt

# -------------------------------
# 레이어 베이스
# -------------------------------
class BackgroundLayer:
    def __init__(self, depth: int = 0):
        self.depth = depth  # 작을수록 뒤, 클수록 앞(렌더 순서에 사용)

    def update(self, dt: float, camera: Camera):
        pass

    def render(self, surface: pygame.Surface, camera: Camera):
        raise NotImplementedError

# -------------------------------
# 단색 도색 레이어(하늘색 같은 기본 배경)
# -------------------------------
class ColorLayer(BackgroundLayer):
    def __init__(self, color: Tuple[int, int, int], depth: int = -1000):
        super().__init__(depth)
        self.color = color

    def render(self, surface: pygame.Surface, camera: Camera):
        surface.fill(self.color)

# -------------------------------
# 이미지 타일 레이어: 패럴랙스 + 무한 타일링
# -------------------------------
class TiledImageLayer(BackgroundLayer):
    def __init__(
        self,
        image: pygame.Surface,
        parallax: Tuple[float, float] = (0.5, 0.5),
        scroll_speed: Tuple[float, float] = (0.0, 0.0),
        depth: int = 0,
        alpha: Optional[int] = None,
    ):
        """
        parallax: (px, py) 카메라 이동에 대한 시차 계수(0=고정 하늘, 1=월드와 같은 속도)
        scroll_speed: 자가 스크롤(구름 떠다니는 느낌)
        """
        super().__init__(depth)
        self.base_image = image.convert_alpha()
        self.img_w, self.img_h = self.base_image.get_size()
        self.parallax = Vec2(parallax)
        self.scroll_speed = Vec2(scroll_speed)
        self.offset = Vec2(0, 0)  # 타일 시작 오프셋
        if alpha is not None:
            self.base_image.set_alpha(alpha)

    def update(self, dt: float, camera: Camera):
        # 자가 스크롤
        self.offset += self.scroll_speed * dt

    def render(self, surface: pygame.Surface, camera: Camera):
        sw, sh = surface.get_size()

        # 카메라에 의한 시차 오프셋(반대로 움직이므로 -camera.pos)
        parallax_offset = -Vec2(camera.pos.x * self.parallax.x,
                                camera.pos.y * self.parallax.y)

        total_offset = self.offset + parallax_offset

        # 현재 타일링 시작점 (음수 modulo 처리)
        start_x = int(total_offset.x) % self.img_w - self.img_w
        start_y = int(total_offset.y) % self.img_h - self.img_h

        # 화면을 완전히 덮도록 타일 렌더
        x = start_x
        while x < sw:
            y = start_y
            while y < sh:
                surface.blit(self.base_image, (x, y))
                y += self.img_h
            x += self.img_w

# -------------------------------
# 고정 이미지 레이어: 화면 기준 배치(예: UI성 별, 안개)
# -------------------------------
class ScreenSpaceImageLayer(BackgroundLayer):
    def __init__(self, image: pygame.Surface, pos: Tuple[int, int], depth: int = 1000, alpha: Optional[int] = None):
        super().__init__(depth)
        self.image = image.convert_alpha()
        if alpha is not None:
            self.image.set_alpha(alpha)
        self.pos = pos

    def render(self, surface: pygame.Surface, camera: Camera):
        surface.blit(self.image, self.pos)

# -------------------------------
# 배경 매니저: 레이어 등록/정렬/일괄 Update/Render
# -------------------------------
class BackgroundManager:
    def __init__(self):
        self.layers: List[BackgroundLayer] = []
        self._dirty = True  # depth가 바뀌면 정렬 필요

    def add(self, layer: BackgroundLayer):
        self.layers.append(layer)
        self._dirty = True

    def sort_if_needed(self):
        if self._dirty:
            # depth 낮은 것부터 먼저 그리기
            self.layers.sort(key=lambda L: L.depth)
            self._dirty = False

    def update(self, dt: float, camera: Camera):
        self.sort_if_needed()
        for layer in self.layers:
            layer.update(dt, camera)

    def render(self, surface: pygame.Surface, camera: Camera):
        # 이미 update에서 정렬 보장하지만 안전빵
        self.sort_if_needed()
        for layer in self.layers:
            layer.render(surface, camera)

# -------------------------------
# 간단 데모 루프
# -------------------------------
if __name__ == "__main__":
    pygame.init()
    W, H = 1280, 720
    screen = pygame.display.set_mode((W, H))
    clock = pygame.time.Clock()

    # 샘플 텍스처 만들기(이미지 파일 없어도 돌아가게)
    def make_checker(w, h, cell=32, c1=(30, 30, 40), c2=(35, 35, 50)):
        surf = pygame.Surface((w, h), pygame.SRCALPHA)
        for y in range(0, h, cell):
            for x in range(0, w, cell):
                r = ((x // cell) + (y // cell)) % 2
                color = c1 if r == 0 else c2
                pygame.draw.rect(surf, color, (x, y, cell, cell))
        return surf

    def make_clouds(w, h):
        surf = pygame.Surface((w, h), pygame.SRCALPHA)
        for i in range(20):
            r = pygame.Rect(0, 0, 220, 90)
            r.center = (pygame.rand.randint(0, w), pygame.rand.randint(0, h)) if hasattr(pygame, "rand") else (i*60 % w, (i*33) % h)
            pygame.draw.ellipse(surf, (255, 255, 255, 90), r)
        return surf

    # 텍스처(여기선 패턴 생성)
    far_tex   = make_checker(512, 512, cell=64, c1=(10,15,25), c2=(12,18,28))    # 가장 먼 배경
    mid_tex   = make_checker(512, 512, cell=32, c1=(18,23,35), c2=(22,28,40))    # 중간
    near_tex  = make_checker(512, 512, cell=16, c1=(28,35,55), c2=(30,38,60))    # 가까운
    cloud_tex = make_checker(512, 256, cell=64, c1=(255,255,255,25), c2=(255,255,255,0))  # 구름 느낌

    # 레이어 구성
    cam = Camera((0, 0))
    bg = BackgroundManager()
    bg.add(ColorLayer((8, 12, 20), depth=-10_000))  # 하늘색 같은 바닥 도색
    bg.add(TiledImageLayer(far_tex,  parallax=(0.2, 0.0), scroll_speed=(0, 0),     depth=-100))
    bg.add(TiledImageLayer(mid_tex,  parallax=(0.5, 0.0), scroll_speed=(-5, 0),    depth=-50))
    bg.add(TiledImageLayer(near_tex, parallax=(0.85, 0.0), scroll_speed=(-15, 0),  depth=-10))
    bg.add(TiledImageLayer(cloud_tex, parallax=(0.1, 0.0), scroll_speed=(-8, 0),   depth=-200, alpha=160))

    # 화면 공간(카메라 무시) 장식 예시
    vignette = pygame.Surface((W, H), pygame.SRCALPHA)
    pygame.draw.rect(vignette, (0,0,0,80), vignette.get_rect(), border_radius=0)
    bg.add(ScreenSpaceImageLayer(vignette, (0, 0), depth=10_000))

    running = True
    speed = 260  # 카메라 이동 속도(px/s)

    while running:
        dt = clock.tick(144) / 1000.0  # 초 단위 delta
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                running = False

        # 입력 -> 카메라 속도
        keys = pygame.key.get_pressed()
        cam.vel.xy = (0, 0)
        if keys[pygame.K_a] or keys[pygame.K_LEFT]:
            cam.vel.x = -speed
        if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
            cam.vel.x = +speed
        if keys[pygame.K_w] or keys[pygame.K_UP]:
            cam.vel.y = -speed
        if keys[pygame.K_s] or keys[pygame.K_DOWN]:
            cam.vel.y = +speed

        # Update
        cam.update(dt)
        bg.update(dt, cam)

        # Render
        bg.render(screen, cam)

        # 디버그 텍스트
        pygame.display.set_caption(f"Camera: {cam.pos.x:.1f}, {cam.pos.y:.1f}  |  dt={dt*1000:.2f}ms")
        pygame.display.flip()

    pygame.quit()

In [ ]:
# assets_model.py
# pygame 2.x / Python 3.9+
from __future__ import annotations
import os, json, glob, threading
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional, Tuple

import pygame

# ---------- 타입 ----------
LoaderFn = Callable[[dict], Any]  # spec(dict)를 받아 로드한 객체 반환
SubscriberFn = Callable[[str, dict], None]  # (event_name, payload)

@dataclass
class AssetRecord:
    key: str
    type: str
    path: Optional[str] = None
    obj: Any = None
    tags: set = field(default_factory=set)
    refs: int = 0
    meta: dict = field(default_factory=dict)
    scaled_cache: Dict[Tuple[int, int], Any] = field(default_factory=dict)  # (w,h) -> surface/font

# ---------- Model ----------
class AssetModel:
    """
    MVC의 Model: 에셋 상태/캐시/이벤트를 관리.
    - 모든 자원은 dict(self.assets)로 관리: key -> AssetRecord
    - 로더는 확장자/타입별 registry에서 자동 선택
    - 매니페스트(JSON)로 일괄 preload 가능
    """
    def __init__(self, base_dir: str = "."):
        self.base_dir = os.path.abspath(base_dir)
        self.assets: Dict[str, AssetRecord] = {}
        self.loaders_by_ext: Dict[str, LoaderFn] = {}
        self.loaders_by_type: Dict[str, LoaderFn] = {}
        self.subscribers: List[SubscriberFn] = []
        self._lock = threading.RLock()

        # 기본 로더 등록
        self._register_default_loaders()

    # ----- 구독(옵저버) -----
    def subscribe(self, fn: SubscriberFn):
        with self._lock:
            self.subscribers.append(fn)

    def _emit(self, event: str, **payload):
        for fn in list(self.subscribers):
            try:
                fn(event, payload)
            except Exception:
                pass  # 구독자 에러는 모델 동작에 영향 X

    # ----- 로더 등록 -----
    def register_loader_for_ext(self, ext: str, loader: LoaderFn):
        self.loaders_by_ext[ext.lower().lstrip(".")] = loader

    def register_loader_for_type(self, typ: str, loader: LoaderFn):
        self.loaders_by_type[typ.lower()] = loader

    def _register_default_loaders(self):
        # 이미지
        def load_image(spec: dict):
            path = self._abs(spec["path"])
            surf = pygame.image.load(path)
            if spec.get("alpha", True):
                surf = surf.convert_alpha()
            else:
                surf = surf.convert()
            if "colorkey" in spec:
                surf.set_colorkey(tuple(spec["colorkey"]))
            return surf

        # 사운드
        def load_sound(spec: dict):
            path = self._abs(spec["path"])
            return pygame.mixer.Sound(path)

        # 폰트 (size 필수)
        def load_font(spec: dict):
            path = spec.get("path")
            size = int(spec["size"])
            if path is None or path.lower() == "default":
                return pygame.font.Font(None, size)
            return pygame.font.Font(self._abs(path), size)

        # 텍스트/JSON 등 일반 파일
        def load_text(spec: dict):
            with open(self._abs(spec["path"]), "r", encoding=spec.get("encoding", "utf-8")) as f:
                return f.read()

        def load_json(spec: dict):
            with open(self._abs(spec["path"]), "r", encoding="utf-8") as f:
                return json.load(f)

        # 확장자 기반
        for e in ["png", "jpg", "jpeg", "bmp", "gif", "webp"]:
            self.register_loader_for_ext(e, load_image)
        for e in ["ogg", "wav", "mp3"]:
            self.register_loader_for_ext(e, load_sound)
        self.register_loader_for_ext("txt", load_text)
        self.register_loader_for_ext("json", load_json)

        # 타입 기반
        self.register_loader_for_type("image", load_image)
        self.register_loader_for_type("sound", load_sound)
        self.register_loader_for_type("font", load_font)
        self.register_loader_for_type("text", load_text)
        self.register_loader_for_type("json", load_json)

    # ----- 유틸 -----
    def _abs(self, path: str) -> str:
        if os.path.isabs(path):
            return path
        return os.path.abspath(os.path.join(self.base_dir, path))

    def exists(self, key: str) -> bool:
        return key in self.assets

    def get(self, key: str) -> Any:
        rec = self.assets.get(key)
        return rec.obj if rec else None

    def ref(self, key: str):
        with self._lock:
            if key in self.assets:
                self.assets[key].refs += 1

    def unref(self, key: str):
        with self._lock:
            if key in self.assets and self.assets[key].refs > 0:
                self.assets[key].refs -= 1

    def keys_by_tag(self, *tags: str) -> List[str]:
        out = []
        for k, rec in self.assets.items():
            if set(tags).issubset(rec.tags):
                out.append(k)
        return out

    # ----- 로드/언로드 -----
    def load(self, key: str, spec: dict) -> Any:
        """
        spec 예시:
        - {"type":"image", "path":"img/player.png", "alpha":True, "tags":["sprite","player"]}
        - {"path":"sound/hit.wav"}  # 확장자로 타입 추론
        - {"type":"font", "path":"C:/Windows/Fonts/malgun.ttf", "size":24}
        """
        with self._lock:
            if key in self.assets and self.assets[key].obj is not None:
                return self.assets[key].obj  # 이미 로드됨

            typ = (spec.get("type") or "").lower()
            path = spec.get("path")

            loader = None
            if typ and typ in self.loaders_by_type:
                loader = self.loaders_by_type[typ]
            elif path:
                ext = os.path.splitext(path)[1].lower().lstrip(".")
                loader = self.loaders_by_ext.get(ext)

            if loader is None:
                raise ValueError(f"[AssetModel] 로더를 찾을 수 없음: spec={spec}")

            obj = loader(spec)

            rec = self.assets.get(key) or AssetRecord(key=key, type=typ or "auto")
            rec.obj = obj
            rec.path = path
            rec.tags |= set(spec.get("tags", []))
            rec.meta.update({k: v for k, v in spec.items() if k not in ("tags")})
            self.assets[key] = rec

            self._emit("asset_loaded", key=key, type=rec.type, path=path, tags=list(rec.tags))
            return obj

    def alias(self, alias_key: str, target_key: str):
        """같은 객체를 다른 key로 공유(얕은 별칭)."""
        with self._lock:
            if target_key not in self.assets:
                raise KeyError(f"target_key not loaded: {target_key}")
            base = self.assets[target_key]
            self.assets[alias_key] = AssetRecord(
                key=alias_key, type=base.type, path=base.path, obj=base.obj,
                tags=set(base.tags), refs=base.refs, meta=dict(base.meta)
            )
            self._emit("asset_aliased", alias=alias_key, target=target_key)

    def unload(self, key: str, force: bool=False) -> bool:
        """refs>0이면 기본적으로 언로드 안 함. force=True로 강제."""
        with self._lock:
            rec = self.assets.get(key)
            if not rec:
                return False
            if rec.refs > 0 and not force:
                return False
            # pygame 객체 해제는 가비지콜렉터에 맡김. (Sound는 stop 등 필요시 처리)
            del self.assets[key]
            self._emit("asset_unloaded", key=key)
            return True

    def purge_unused(self) -> int:
        """refs==0 인 에셋 정리."""
        with self._lock:
            to_del = [k for k, r in self.assets.items() if r.refs == 0]
            for k in to_del:
                del self.assets[k]
            if to_del:
                self._emit("assets_purged", count=len(to_del), keys=to_del)
            return len(to_del)

    # ----- 배치 로드 -----
    def load_dir(self, pattern: str, tag: Optional[str]=None, prefix: str=""):
        """
        예: model.load_dir('assets/img/**/*.png', tag='img', prefix='img/')
        key는 prefix+상대경로(확장자 포함)로 저장.
        """
        base = self._abs(".")
        for path in glob.glob(self._abs(pattern), recursive=True):
            rel = os.path.relpath(path, self.base_dir).replace("\\", "/")
            key = prefix + rel
            ext = os.path.splitext(path)[1].lower().lstrip(".")
            loader = self.loaders_by_ext.get(ext)
            if not loader:
                continue
            self.load(key, {"path": rel, "tags": [tag] if tag else []})

    # ----- 매니페스트 -----
    def save_manifest(self, out_path: str):
        data = []
        for k, r in self.assets.items():
            data.append({
                "key": k, "type": r.type, "path": r.path,
                "tags": sorted(list(r.tags)), "meta": r.meta
            })
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump({"base_dir": self.base_dir, "assets": data}, f, ensure_ascii=False, indent=2)

    def load_manifest(self, manifest_path: str, preload: bool=True):
        with open(manifest_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if "base_dir" in data:
            self.base_dir = data["base_dir"]
        for item in data.get("assets", []):
            key = item["key"]
            spec = {k: v for k, v in item.items() if k != "key"}
            if preload:
                self.load(key, spec)
            else:
                # 기록만 해두고 필요 시 lazy load하려면 빈 레코드 저장
                self.assets[key] = AssetRecord(
                    key=key, type=spec.get("type", "auto"), path=spec.get("path"),
                    tags=set(spec.get("tags", [])), meta=spec.get("meta", {})
                )
        self._emit("manifest_loaded", count=len(data.get("assets", [])))

    # ----- 이미지 관리 -----
    # ------------------------------
    # 이미지 크기 관리
    # ------------------------------
    def get_image_scaled(self, key: str, size: Tuple[int, int]) -> pygame.Surface:
        """
        key: 원본 이미지 키
        size: (width, height) 튜플
        """
        rec = self.assets.get(key)
        if not rec:
            raise KeyError(f"[AssetModel] 이미지 '{key}' 로드 안 됨")

        if rec.type != "image":
            raise TypeError(f"[AssetModel] '{key}'는 이미지가 아님 (type={rec.type})")

        # 캐시 있으면 바로 반환
        if size in rec.scaled_cache:
            return rec.scaled_cache[size]

        surf = rec.obj
        scaled = pygame.transform.smoothscale(surf, size)
        rec.scaled_cache[size] = scaled
        return scaled

    # ------------------------------
    # 텍스트(폰트) 크기 관리
    # ------------------------------
    def get_font_resized(self, key: str, size: int) -> pygame.font.Font:
        """
        key: 원본 폰트 키
        size: 원하는 폰트 사이즈(px)
        """
        rec = self.assets.get(key)
        if not rec:
            raise KeyError(f"[AssetModel] 폰트 '{key}' 로드 안 됨")

        if rec.type != "font":
            raise TypeError(f"[AssetModel] '{key}'는 폰트가 아님 (type={rec.type})")

        cache_key = (size, 0)  # 폰트는 (size,0) 같은 튜플로 캐싱
        if cache_key in rec.scaled_cache:
            return rec.scaled_cache[cache_key]

        # path가 None이면 기본 폰트
        path = rec.path if rec.path and rec.path.lower() != "default" else None
        new_font = pygame.font.Font(path, size)
        rec.scaled_cache[cache_key] = new_font
        return new_font